# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.110967e+02     1.561077e+00
 * time: 0.8085858821868896
     1     1.093044e+01     8.456962e-01
 * time: 2.238671064376831
     2    -1.126373e+01     9.539656e-01
 * time: 2.917288064956665
     3    -3.406989e+01     7.095714e-01
 * time: 3.936923027038574
     4    -4.749263e+01     5.436764e-01
 * time: 4.989543914794922
     5    -5.704709e+01     2.325354e-01
 * time: 6.001209020614624
     6    -5.994957e+01     1.223554e-01
 * time: 6.674109935760498
     7    -6.099807e+01     4.321675e-02
 * time: 7.36518406867981
     8    -6.137786e+01     6.605933e-02
 * time: 8.051923990249634
     9    -6.163496e+01     3.901783e-02
 * time: 8.73885703086853
    10    -6.183567e+01     2.594680e-02
 * time: 9.444921016693115
    11    -6.199801e+01     1.714071e-02
 * time: 10.143418073654175
    12    -6.206154e+01     1.620563e-02
 * time: 10.808022022247314
    13    -6.212654e+01     1.436242e-02
 * time: 11.474538087844849
    1

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671064
    AtomicLocal         -18.8557705
    AtomicNonlocal      14.8522676
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666459741